In [ ]:
from google.colab import drive

drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
game = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/project/vgames2.csv')

In [ ]:
game.info()

In [ ]:
game.shape

In [ ]:
game.isnull().sum()

In [ ]:
game.groupby('Genre').count().index

In [ ]:
game.groupby('Year').count().index

### 데이터 전처리
기본
- 결측치 처리
- 데이터 형변환
- 행과 열 정리

In [ ]:
#year 결측치 처리

In [ ]:
game = game.drop(columns={'Unnamed: 0'},axis=1)

In [ ]:
game.loc[game['Year'].isnull()]

In [ ]:
year_list = list(game['Year'].fillna(0).astype('int32'))

In [ ]:
for i in range(len(year_list)) :
  if 0<year_list[i]<20 :
    year_list[i] = 2000+year_list[i]
  elif 20<=year_list[i]<100 :
    year_list[i] = 1900+year_list[i]
  else : continue

In [ ]:
game['Year'] = pd.Series(year_list)

In [ ]:
set(list(game['Year']))

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def make_plot (s,l) :
  tmp = [float(i.rstrip(l)) for i in game[s] if i[-1] == l]
  print(tmp)
  plt.figure(figsize=(30,10))
  plt.hist(tmp)
  plt.show()

In [ ]:
make_plot('NA_Sales','K')

In [ ]:
make_plot('NA_Sales','M')

위의 결과를 통해 'K'는 대체로 0~500사이의 숫자와 함께 나오고 'M'은 소숫점 자릿수와 함께 나온다. 알파벳이 붙지 않은채 나온 결과들도 대부분 소숫점 자릿수로 표현되는 것으로 보아 K에 1000을 나누어 단위를 M로 통일시키는 것이 좋을 것으로 판단하였다.

In [ ]:
 #K.M을 삭제하고 스케일링 작업
 def convert_number (s) :
   game.loc[game[s].str.contains('M'), s] = game.loc[game[s].str.contains('M'), s].str.replace('M','')
   game.loc[game[s].str.contains('K'), s] = game.loc[game[s].str.contains('K'), s].str.replace('K','').astype('float').apply(lambda i : i/1000)
   game[s] = game[s].astype('float')
   game[s] = game[s]

In [ ]:
convert_number('NA_Sales')

In [ ]:
convert_number('JP_Sales'), convert_number('EU_Sales'), convert_number('Other_Sales')

In [ ]:
game.head(10)

In [ ]:
game.dtypes

In [ ]:
"""#2. def isnull(x):
  if x[-1] == 'K':
    return x
  return None

tmp1_new = tmp1.apply(isnull)
tmp1_new2 = tmp1_new.dropna()"""

#1. tmp1_new = tmp1.apply(lambda x : None if x[-1] != 'M' else x)
#왜 다 None이 출력될까?

### 데이터 분석하기 1 : 나라마다 선호하는 게임이 다른가?
- 귀무가설 : 나라와 선호하는 게임의 장르는 관련이 없다
- 대안가설 : 나라와 선호하는 게임의 장르는 관련이 있다

In [ ]:
game.head(10)

In [ ]:
#나라별 장르별 매출액 파이차트로 나타내기
NA = game[['Genre','NA_Sales']]
grouped_NA = NA.groupby('Genre').sum()

In [ ]:
grouped_NA.index

In [ ]:
EU = game[['Genre','EU_Sales']]
grouped_EU = EU.groupby('Genre').sum()

In [ ]:
JP = game[['Genre','JP_Sales']]
grouped_JP = JP.groupby('Genre').sum()

In [ ]:
Other = game[['Genre','Other_Sales']]
grouped_Other = Other.groupby('Genre').sum()

In [ ]:
fig = plt.figure(figsize=(20,20))
ax1 = fig.add_subplot(1,4,1)
ax1.pie(grouped_NA['NA_Sales'],labels=grouped_NA.index,autopct='%.1f%%',wedgeprops={'width':0.7,'edgecolor':'w','linewidth':5},textprops={'size': 10},startangle=260)
ax1.set_title('North America Sales(Genre)',fontsize='18',fontweight='bold')
#plt.pie 의 output은 'patch', 'texts(patch_name)', percentage(pcts)이다

ax2 = fig.add_subplot(1,4,2)
ax2.pie(grouped_EU['EU_Sales'],labels=grouped_EU.index,autopct='%.1f%%',wedgeprops={'width':0.7,'edgecolor':'w','linewidth':5},textprops={'size': 10},startangle=260)
ax2.set_title('EU Sales(Genre)',fontsize='18',fontweight='bold')

ax3 = fig.add_subplot(1,4,3)
ax3.pie(grouped_JP['JP_Sales'],labels=grouped_JP.index,autopct='%.1f%%',wedgeprops={'width':0.7,'edgecolor':'w','linewidth':5},textprops={'size': 10},startangle=260)
ax3.set_title('JP Sales(Genre)',fontsize='18',fontweight='bold')

ax4 = fig.add_subplot(1,4,4)
ax4.pie(grouped_Other['Other_Sales'],labels=grouped_Other.index,autopct='%.1f%%',wedgeprops={'width':0.7,'edgecolor':'w','linewidth':5},textprops={'size': 10},startangle=260)
ax4.set_title('Other Sales(Genre)',fontsize='18',fontweight='bold')

plt.tight_layout()

In [ ]:
print(grouped_NA.sort_values(by='NA_Sales',ascending=False),end='\n'*5)
print(grouped_EU.sort_values(by='EU_Sales',ascending=False), end='\n'*5)
print(grouped_JP.sort_values(by='JP_Sales',ascending=False), end='\n'*5)
print(grouped_Other.sort_values(by='Other_Sales',ascending=False), end='\n'*5)

단순히 시각적으로 확인했을 때와 출고량이 높은 5개의 게임을 각 나라별로 비교했을 때,  북아메리카와 유럽, 기타국가의 판매량은 유사하지만, 일본의 판매량만 다른 나라와 확연한 차이를 보인다

1. 북아메리카 출고량(매출기준) 상위 5위 :
Action
Sports
Shooter
Platform
Misc

2. 유럽 출고량(매출기준) 상위 5위 :
Action
Sports
Shooter
Racing
Misc

3. 기타국가 출고량(매출기준) 상위 5위 :
Action
Sports
Shooter
Racing
Misc

4. 일본 출고량(매출기준) 상위 5위 :
Role-Playing 
Action   
Sports 
Platform 
Misc